In [ ]:
%matplotlib tk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Check which Python interpreter and kernel you're using
import sys
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")
print(f"Numpy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")


In [ ]:
basename = "1ms_ramp_3A"; time_start = 0.5e-3; time_end = 1.5e-3
# basename = "slice_pa_data"; time_start = 1.5e-3; time_end = 3.0e-3


In [ ]:
dfs = [pd.DataFrame() for _ in range(4)]

# Load the trigger file to determine number of slices
trig_df = pd.read_csv(f"data/{basename}_trig.csv", sep=r"\s+", header=None)
num_slices = len(trig_df)

# Load the files
for i in range(4):
  dfs[i] = pd.read_csv(f"data/{basename}_bd_{i}.csv", sep=r"\s+", header=None)

# Initialize the array
adc_array = dfs[0].to_numpy()

# Append each subsequent DataFrame to the array (in the second dimension)
for i in range(1, 4):
  adc_array = np.concatenate((adc_array, dfs[i].to_numpy()), axis=1)

# Convert ADC values to amps (-2^15 to 2^15 -> -5.0 to 5.0 A)
adc_array = adc_array * (5.0 / 32768)

# Calculate slice_size based on total samples and number of slices
total_samples = adc_array.shape[0]
slice_size = total_samples // num_slices
num_channels = adc_array.shape[1]

print(f"Detected {num_slices} slices from trigger file")
print(f"Total samples: {total_samples}, Slice size: {slice_size}")
print(f"Number of channels: {num_channels}")


In [ ]:
# Calculate statistics for all channels and slices
time_slice = np.arange(1, slice_size + 1) * 20e-6

# Find the indices corresponding to the time range
start_idx = int(time_start / (time_slice[1] - time_slice[0]))
end_idx = int(time_end / (time_slice[1] - time_slice[0]))

# Calculate mean and standard deviation for each (channel, slice) pair
all_means = []
all_stds = []
all_channels = []
all_slices = []

for slice_idx in range(num_slices):
  slice_start = slice_idx * slice_size + start_idx
  slice_end = slice_idx * slice_size + end_idx
  
  for ch_idx in range(num_channels):
    # Get data for this channel and slice in the time range
    channel_slice_data = adc_array[slice_start:slice_end, ch_idx]
    
    # Calculate mean and standard deviation
    mean_current = np.mean(channel_slice_data)
    std_current = np.std(channel_slice_data)
    
    all_means.append(mean_current)
    all_stds.append(std_current)
    all_channels.append(ch_idx)
    all_slices.append(slice_idx)

# Define hover function for scatter plot
def on_scatter_hover(event, ax, scatter, all_channels, all_slices, annotation):
  if event.inaxes != ax:
    annotation.set_visible(False)
    event.canvas.figure.canvas.draw_idle()
    return
  
  # Check if hovering over a point
  cont, ind = scatter.contains(event)
  if cont:
    # Get the index of the first point under cursor
    point_idx = ind['ind'][0]
    channel = all_channels[point_idx]
    slice_idx = all_slices[point_idx]
    
    # Update annotation
    annotation.set_text(f'Ch {channel}, Slice {slice_idx}')
    annotation.xy = (event.xdata, event.ydata)
    annotation.set_visible(True)
  else:
    annotation.set_visible(False)
  
  event.canvas.figure.canvas.draw_idle()

# Create figure with 2x2 subplots
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Convert to numpy arrays for easier indexing
all_means_arr = np.array(all_means)
all_stds_arr = np.array(all_stds)
all_channels_arr = np.array(all_channels)
all_slices_arr = np.array(all_slices)

# Top-left: Color by channel (log scale)
scatter1 = ax1.scatter(all_means_arr, all_stds_arr, c=all_channels_arr, 
                       cmap='tab20', marker='o', s=50, alpha=0.7)
ax1.set_xlabel('Mean Current (A)')
ax1.set_ylabel('Standard Deviation (A, log scale)')
ax1.set_yscale('log')
ax1.set_title(f'STD vs Mean - Color by Channel (Log)\n(Time range: {time_start*1000:.1f} ms to {time_end*1000:.1f} ms)')
ax1.grid(True)
cbar1 = plt.colorbar(scatter1, ax=ax1, label='Channel')

# Create annotation for top-left subplot
annotation1 = ax1.annotate('', xy=(0,0), xytext=(20,20), textcoords='offset points',
                           bbox=dict(boxstyle='round', fc='yellow', alpha=0.9),
                           arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'),
                           visible=False, fontsize=10, weight='bold')

# Top-right: Color by slice (log scale)
scatter2 = ax2.scatter(all_means_arr, all_stds_arr, c=all_slices_arr,
                       cmap='viridis', marker='o', s=50, alpha=0.7)
ax2.set_xlabel('Mean Current (A)')
ax2.set_ylabel('Standard Deviation (A, log scale)')
ax2.set_yscale('log')
ax2.set_title(f'STD vs Mean - Color by Slice (Log)\n(Time range: {time_start*1000:.1f} ms to {time_end*1000:.1f} ms)')
ax2.grid(True)
cbar2 = plt.colorbar(scatter2, ax=ax2, label='Slice')

# Create annotation for top-right subplot
annotation2 = ax2.annotate('', xy=(0,0), xytext=(20,20), textcoords='offset points',
                           bbox=dict(boxstyle='round', fc='yellow', alpha=0.9),
                           arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'),
                           visible=False, fontsize=10, weight='bold')

# Bottom-left: Color by channel (linear scale)
scatter3 = ax3.scatter(all_means_arr, all_stds_arr, c=all_channels_arr, 
                       cmap='tab20', marker='o', s=50, alpha=0.7)
ax3.set_xlabel('Mean Current (A)')
ax3.set_ylabel('Standard Deviation (A)')
ax3.set_title(f'STD vs Mean - Color by Channel (Linear)\n(Time range: {time_start*1000:.1f} ms to {time_end*1000:.1f} ms)')
ax3.grid(True)
cbar3 = plt.colorbar(scatter3, ax=ax3, label='Channel')

# Create annotation for bottom-left subplot
annotation3 = ax3.annotate('', xy=(0,0), xytext=(20,20), textcoords='offset points',
                           bbox=dict(boxstyle='round', fc='yellow', alpha=0.9),
                           arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'),
                           visible=False, fontsize=10, weight='bold')

# Bottom-right: Color by slice (linear scale)
scatter4 = ax4.scatter(all_means_arr, all_stds_arr, c=all_slices_arr,
                       cmap='viridis', marker='o', s=50, alpha=0.7)
ax4.set_xlabel('Mean Current (A)')
ax4.set_ylabel('Standard Deviation (A)')
ax4.set_title(f'STD vs Mean - Color by Slice (Linear)\n(Time range: {time_start*1000:.1f} ms to {time_end*1000:.1f} ms)')
ax4.grid(True)
cbar4 = plt.colorbar(scatter4, ax=ax4, label='Slice')

# Create annotation for bottom-right subplot
annotation4 = ax4.annotate('', xy=(0,0), xytext=(20,20), textcoords='offset points',
                           bbox=dict(boxstyle='round', fc='yellow', alpha=0.9),
                           arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'),
                           visible=False, fontsize=10, weight='bold')

# Connect hover events
fig.canvas.mpl_connect('motion_notify_event',
                       lambda event: on_scatter_hover(event, ax1, scatter1, all_channels, all_slices, annotation1))
fig.canvas.mpl_connect('motion_notify_event',
                       lambda event: on_scatter_hover(event, ax2, scatter2, all_channels, all_slices, annotation2))
fig.canvas.mpl_connect('motion_notify_event',
                       lambda event: on_scatter_hover(event, ax3, scatter3, all_channels, all_slices, annotation3))
fig.canvas.mpl_connect('motion_notify_event',
                       lambda event: on_scatter_hover(event, ax4, scatter4, all_channels, all_slices, annotation4))

plt.tight_layout()
plt.show()
